In [85]:
from pathlib import Path
import pandas as pd

data_dir = Path.cwd() / "data"
jobs = ["IOPFPlanets20220810", "IOPFPlanets20220820"]

argfiles = [argfile for job in jobs for argfile in (data_dir / job).glob("*/args.json")]
sims_records = pd.DataFrame.from_records(pd.read_json(argfile, typ="series") for argfile in argfiles)
sims_paths = pd.DataFrame.from_records({"run": argfile.parent.name, "path": argfile.parent} for argfile in argfiles)
sims_df = pd.concat([sims_records, sims_paths], axis="columns")

In [86]:
import rebound

final_cache = {}
for i, rebound_path in (sims_df["path"] / sims_df["rebound_archive"]).items():
    sa = rebound.SimulationArchive(str(rebound_path))
    final = sa[-1]
    final_cache[sims_df.at[i, "run"]] = {
        "sa": sa,
        "final": final
    }

/home/alchzh/cassum-2022-project-21/workspace/src/rebound/rebound/simulationarchive.py:146: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


In [91]:
import numpy as np

NaN = float("NaN")

def FINAL_STATS(sim):
    final = final_cache[sim["run"]]["final"]
    
    stats = np.array([(p.m, p.a, p.P) for p in final.particles[1:]]).T
    N = len(stats[0])
    M_TOT = stats[0].sum()

    top2 = stats[:, np.argsort(stats[0])[-1:-3:-1]]
    if N < 2:
        top2 = np.column_stack([top2, np.full_like(top2, NaN)])
    
    m, a, P = top2  

    return pd.Series({
        "FINAL_N": N,
        "FINAL_M_TOT": M_TOT,

        "FINAL_M_PRI": m[0],
        "FINAL_P_PRI": P[0],
        "FINAL_A_PRI": a[0],

        "FINAL_M_SEC": m[1],
        "FINAL_P_SEC": P[1],
        "FINAL_A_SEC": a[1],

        "FINAL_M_RAT_PRI_TOT": m[0] / M_TOT,
        "FINAL_M_RAT_SEC_TOT": m[1] / M_TOT,
        "FINAL_P_RAT_PRI_SEC": m[1] / M_TOT,
    })

final_df = sims_df.apply(FINAL_STATS, axis=1)
final_df["FINAL_N"] = final_df["FINAL_N"].astype(np.int_)
sims_ext_df = sims_df.join(final_df)
sims_ext_df

,seed,t_end,N_end,N_enddelay,store_dt,n_p,code,a_in,a_out,alpha,...,FINAL_M_TOT,FINAL_M_PRI,FINAL_P_PRI,FINAL_A_PRI,FINAL_M_SEC,FINAL_P_SEC,FINAL_A_SEC,FINAL_M_RAT_PRI_TOT,FINAL_M_RAT_SEC_TOT,FINAL_P_RAT_PRI_SEC
0,15161,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.099743,0.215075,1.185814e-06,0.128945,0.255233,0.583821,0.394911,0.394911
1,15161,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000001,0.124418,0.249224,1.265649e-06,0.098649,0.213499,0.440965,0.421499,0.421499
2,19231,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.127712,0.253603,1.228405e-06,0.089109,0.199503,0.590905,0.409095,0.409095
3,19231,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000001,0.122703,0.246928,9.271302e-07,0.091165,0.202559,0.453486,0.308762,0.308762
4,20223,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.106459,0.224624,3.669984e-07,0.145382,0.276486,0.830260,0.122221,0.122221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,81988,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000001,0.108763,0.227853,9.105758e-07,0.136027,0.264494,0.385354,0.303249,0.303249
60,95704,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.108715,0.227786,5.402646e-07,0.086840,0.196101,0.786794,0.179924,0.179924
61,95704,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.108842,0.227963,7.045865e-07,0.093280,0.205680,0.566822,0.234648,0.234648
62,96616,2000000.0,1,1.0,1000.0,100,rebound,0.22,0.24,-2,...,0.000003,0.000002,0.108680,0.227737,9.467219e-07,0.132854,0.260365,0.673728,0.315286,0.315286


In [117]:
groups = sims_ext_df.groupby(["C_d", "migration_torque"])

mins = groups.min().add_suffix("_MIN")
maxes = groups.max().add_suffix("_MAX")
means = groups.mean().add_suffix("_MEAN")
medians = groups.median().add_suffix("_MEDIAN")

summary = pd.concat([mins, maxes, means, medians], axis=1).filter(like="FINAL")
summary

FINAL_N_MIN  FINAL_M_TOT_MIN  FINAL_M_PRI_MIN  \
C_d  migration_torque                                                  
0.00 False                       2         0.000003         0.000001   
     True                        1         0.000003         0.000002   
0.44 False                       3         0.000003         0.000001   
     True                        3         0.000003         0.000001   

                       FINAL_P_PRI_MIN  FINAL_A_PRI_MIN  FINAL_M_SEC_MIN  \
C_d  migration_torque                                                      
0.00 False                    0.090833         0.202067     3.669984e-07   
     True                     0.108680         0.227737     6.081989e-08   
0.44 False                    0.094793         0.207898     5.537375e-07   
     True                     0.108580         0.227598     3.443561e-07   

                       FINAL_P_SEC_MIN  FINAL_A_SEC_MIN  \
C_d  migration_torque                                     
0.00 False                    0.085038         0.193378   
     True                     0.084560         0.192653   
0.44 False                    0.084367         0.192360   
     True                     0.081545         0.188047   

                       FINAL_M_RAT_PRI_TOT_MIN  FINAL_M_RAT_SEC_TOT_MIN  ...  \
C_d  migration_torque                                                    ...   
0.00 False                            0.472246                 0.122221  ...   
     True                             0.610590                 0.020255  ...   
0.44 False                            0.397199                 0.184411  ...   
     True                             0.335261                 0.114681  ...   

                       FINAL_M_TOT_MEDIAN  FINAL_M_PRI_MEDIAN  \
C_d  migration_torque                                           
0.00 False                       0.000003            0.000002   
     True                        0.000003            0.000002   
0.44 False                       0.000003            0.000001   
     True                        0.000003            0.000002   

                       FINAL_P_PRI_MEDIAN  FINAL_A_PRI_MEDIAN  \
C_d  migration_torque                                           
0.00 False                       0.106894            0.225235   
     True                        0.108715            0.227786   
0.44 False                       0.110400            0.230133   
     True                        0.108706            0.227773   

                       FINAL_M_SEC_MEDIAN  FINAL_P_SEC_MEDIAN  \
C_d  migration_torque                                           
0.00 False                   1.141105e-06            0.128547   
     True                    5.343094e-07            0.134359   
0.44 False                   9.313762e-07            0.100344   
     True                    9.484590e-07            0.130618   

                       FINAL_A_SEC_MEDIAN  FINAL_M_RAT_PRI_TOT_MEDIAN  \
C_d  migration_torque                                                   
0.00 False                       0.254708                    0.616070   
     True                        0.262324                    0.770551   
0.44 False                       0.215931                    0.495969   
     True                        0.257436                    0.548325   

                       FINAL_M_RAT_SEC_TOT_MEDIAN  FINAL_P_RAT_PRI_SEC_MEDIAN  
C_d  migration_torque                                                          
0.00 False                               0.380022                    0.380022  
     True                                0.177941                    0.177941  
0.44 False                               0.310176                    0.310176  
     True                                0.315865                    0.315865  

[4 rows x 44 columns]

In [120]:
ORDER = [
    "FINAL_N",
    "FINAL_M_RAT_PRI_TOT",
    "FINAL_M_RAT_SEC_TOT"
]

SUFFIX = [
    "MIN",
    "MAX",
    "MEAN",
    "MEDIAN"
]

COLS = [f"{b}_{s}" for b in ORDER for s in SUFFIX]
summary[COLS].to_csv("summary.tsv", sep="\t")